In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import pipeline
import torch
import numpy as np
import os 
from datetime import datetime
import json



2025-10-28 02:05:29.601268: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-28 02:05:29.680194: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2025-10-28 02:05:34.935589: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
RANDOM_STATE = 42
path_DATA = "../../../data"
csv_path = f"{path_DATA}/spotify_dataset_sin_duplicados_4.csv"

# To compare with  tfidf
zero_shot_path = f"{path_DATA}/zero_outputs"
os.makedirs(zero_shot_path, exist_ok=True)


TESTING = True

if TESTING:
    NROWS = 200
else:
    NROWS = None
    


In [3]:
def get_array(path):
    with open(path, "r") as f:
        array_ = json.load(f)  
    return array_

def get_song_and_target(csv_path, faltantes_path = "faltantes_according_token.json", sample_size=None, path_test_indexes=None):
    df = pd.read_csv(csv_path)
    indices_to_remove = get_array(faltantes_path)
    if indices_to_remove[-1]>df.shape[0]:
        print("You are executing in testing way")
    else:
        df = df.drop(indices_to_remove).reset_index(drop=True)

    if path_test_indexes is not None:
        df_split_indices_test = pd.read_csv(path_test_indexes)
        idx_test = df_split_indices_test["test_index"].values
        df = df.iloc[idx_test].reset_index(drop=True)

    if sample_size is not None:
        df = df.iloc[:sample_size].reset_index(drop=True)
    

    df['original_index'] = df.index # Indices correctos despues de la eliminación
    X = df['text']
    df['Explicit_binary'] = (df['Explicit'].str.lower() == 'yes').astype(int)
    y = df['Explicit_binary']
    return X, y, df['original_index']

In [4]:

# Indices train and test, in this experimentations only is relevante test indexes
df_split_indices = pd.read_csv(f"{zero_shot_path}/split_indices_train.csv")
df_split_indices_test = pd.read_csv(f"{zero_shot_path}/split_indices_test.csv")

X,y, index = get_song_and_target(csv_path,"faltantes_according_token.json", sample_size=NROWS, path_test_indexes=f"{zero_shot_path}/split_indices_test.csv")

print(X.shape)
print(y.shape)
print(index.shape)


(200,)
(200,)
(200,)


# meta-llama/Llama-3.1-8B-Instruct
https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct

In [5]:
import os
import transformers
import torch


model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    pad_token_id=128001
    # pad_token_id` to `eos_token_id`:128001
)

messages = [
    {"role": "system", "content": "You are a expert detecting if a song is explicit or not based on its lyrics. Answer with 'Explicit' or 'Non-Explicit'."},
    {"role": "user", "content": f"song lyric: {X.iloc[1]} Is this song explicit or non-explicit?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


`torch_dtype` is deprecated! Use `dtype` instead!


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


{'role': 'assistant', 'content': 'Explicit.'}


In [6]:

X_test = X
y_test = y

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("index:", index.shape)


X_test shape: (200,)
y_test shape: (200,)
index: (200,)


In [7]:
test_scores = []

i_tested = 0

answers_debug = []
sons_debug = []
for lyric in X_test:
    messages = [
        {"role": "system", "content": "You are an expert detecting if a song is explicit or not based on its lyrics. Answer ONLY with one word: 'Explicit' or 'Non-Explicit'."},
        {"role": "user", "content": f"song lyric: {lyric}"},
    ]
    output = pipeline(messages, max_new_tokens=64)
    # print()
    # answer = output[0]["generated_text"].strip().split()[-1]
    answer = output[0]["generated_text"][-1]['content']
    if TESTING:
        answers_debug.append(answer)
        sons_debug.append(lyric)
    score_explicit = 1 if answer.lower() == "explicit" else 0
    test_scores.append(score_explicit)
    i_tested += 1

    if i_tested == NROWS and TESTING:
        break

test_scores = np.array(test_scores)

# Métricas
f1_test = f1_score(y_test, test_scores)
accuracy_test = accuracy_score(y_test, test_scores)
precision_test = precision_score(y_test, test_scores)
recall_test = recall_score(y_test, test_scores)

print("test_scores length:", len(test_scores))
print("y_indexes length:", len(index))

df_results = pd.DataFrame({
    "original_index": index,
    "explicit_score": test_scores
})
df_results.to_csv(f"{zero_shot_path}/llama_8B_results.csv", index=False)


print(f"Accuracy en test: {accuracy_test:.8f}")
print(f"Precision en test: {precision_test:.8f}")
print(f"Recall en test: {recall_test:.8f}")
print(f"F1 en test: {f1_test:.8f}")


resultados = {
    "accuracy": float(f"{accuracy_test:.8f}"),
    "precision": float(f"{precision_test:.8f}"),
    "recall": float(f"{recall_test:.8f}"),
    "f1": float(f"{f1_test:.8f}")
}

if TESTING:
    test_ou = "test"
else:
    test_ou = ""

ruta_resultados = f"resultados_test_llama_8B{test_ou}.json"
with open(ruta_resultados, "w") as f:
    json.dump(resultados, f, indent=4)

print(f"Resultados guardados en {ruta_resultados}")



You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


test_scores length: 200
y_indexes length: 200
Accuracy en test: 0.84000000
Precision en test: 0.65000000
Recall en test: 0.78000000
F1 en test: 0.70909091
Resultados guardados en resultados_test_llama_8Btest.json


In [8]:
if TESTING:
    import textwrap
    print("Answers debug:", answers_debug)
    print("test_scores:", test_scores)
    print("y true     :", y_test.values)

    print("song 7:", sons_debug[7])
    wrapped_song = textwrap.fill(sons_debug[7], width=80)
    print("song 7:")
    print(wrapped_song)

Answers debug: ['Explicit', 'Explicit', 'Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit.', 'Explicit', 'Explicit', 'Non-Explicit.', 'Explicit', 'Non-Explicit', 'Non-Explicit', 'Explicit', 'Non-Explicit', 'Explicit', 'Non-Explicit', 'Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit.', 'Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Explicit', 'Non-Explicit.', 'Non-Explicit.', 'Explicit', 'Explicit.', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Explicit', 'Explicit', 'Explicit', 'Non-Explicit.', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit', 'Non-Explicit.', 'Non-Explicit', 'Non-Exp